<a href="https://colab.research.google.com/github/sakuronohana/cas_datenanalyse/blob/master/Semesterarbeit/CAS_DA_SA_Payment_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img 
src="https://www.ffhs.ch/templates/ffhs/img/logo@2x.png" width="100"> 
###DaAn, Data Analysis, MAS/CAS Web4B 2018, ZH1, FS19, Dr. Tödtli Beat###

##*Semesterarbeit von Patrik Di Lena*##
#Betrugserkennung Zahlungsverkehr#



##Ziel##
Das Ziel dieser Semesterarbeit ist die Gegenüberstellung von Supervised und Unsupervised Lernverfahren mit eine neuronalen Netzwerk Algorthmus. Dabei kommt  ein nicht gelabelter und ein gelabelter Datensatz aus einem Betrugserkennungssystem zum Einsatz. 

##Aufgabenstellung##
###Erkennung von Betrugszahlungen###
Mittels Unsupervised Learning sollen Betrugszahlungen aus einem nicht gelabelten Datensatz erkannt und die Effiktivität des vorhandenen Fraud-Detection Systems geprüft werden.

###Betrugsanfällige Risikogruppen###
Mit Hilfe von Supervised Learning werden mögliche Risikogruppen ermittelt, welche im Visier von Betrügern stehen.  Auf Basis eines gelabelten Datensatzes  werden die Risikogruppen nach folgenden Merkmalen unterschieden:

*	Geschlecht
*	Alter
*	Zivilstand
*	Nationalität
*	Kontosaldo
*	Wohnort
*	Region

##Datensatz##
Die im Rahmen dieser Semesterarbeit verwendeten Daten entstammen, von ihrer Datenstruktur her, aus einem bereits eingesetzten nicht ML-basierten Betrugserkennungssystem. Die Datenwerte wurde zu Wahrung der datenschutzrechtlichen und bankengesetzlichen Aspekte mit Hilfe eines Python-Scripts künstlich erzeugt und beinhalten somit keine reale Zahlungs- oder Kunden-Informationen.   

Datenselektion:

- 20'800 synthetische Kunden (Vorname, Name, Strasse, Ort, Kanton, Sprachregion, Geschlecht, Alter, Zivilstand,  Nationalität, Kontonummer, Vertragsnummer, Rechtsform)

- 60'000 synthetische Zahlungen (Transaktionsid, ,Erstellungdatum/Zeit, Empfängerkonto, Empfängerbank, Empfängerland, Währung, Betrag, Valuta Datum)








#Erkennung von Betrugszahlungen# 

##Unsupervised Learning mit neuronalen Netzwerken##

###Importieren der ungelabelten Fraud-Zahlungsdaten###

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# Laden der 600000 nicht gelabelten Transaktionsdaten 
trx_data_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/trx_data_ol.csv'

trx_data_ol = pd.read_csv(trx_data_url, delimiter=';')
trx_data_ol.head()



,timestamp,paym_id,send_bc_nr,amount,rcv_bc_nr,rcv_bc_iban,rcv_bc_country,rcv_bc_code,rcv_iban,valuta_date,cust_vertrag_nr,cust_konto_nr,cust_vname,cust_nname,cust_plz,cust_ort,cust_kanton,cust_sprachregion,cust_auth_device
0,01.01.18 08:00,10000010,CHBANK71XXX,15368,JOBANK40XXX,JO02SCBL1260000000018525836101,Jordanien,JO,JO40002096331755419,01.01.18 08:00,76311,CH5300772672529640195,Martin,Bader,6020,Emmenbrücke,LU,de,mtan
1,01.01.18 08:00,10000011,CHBANK71XXX,89137,STBANK68XXX,ST23000200000289355710148,Sao Tome und Principe,ST,ST68002492808312214,01.01.18 20:00,69891,CH5300772670061796126,Margaretha,Fehr,9056,Gais,AR,de,ptan
2,01.01.18 08:01,10000012,CHBANK71XXX,87673,XKBANK43XXX,XK051301001002074155,Kosovo,XK,XK43002422391264290,02.01.18 08:01,78573,CH5300772500037725234,Daniel,Miéville,1253,Vandoeuvres,GE,fr,ptan
3,01.01.18 08:01,10000013,CHBANK71XXX,55941,RSBANK72XXX,RS35105008054113238018,Serbien,RS,RS72002873180573624,02.01.18 20:01,75443,CH5300772732508635138,Andrea,Iseni,6026,Rain,LU,de,mtan
4,01.01.18 08:02,10000014,CHBANK71XXX,88173,LIBANK48XXX,LI0308800000022875748,Liechtenstein,LI,LI48002899580343897,03.01.18 08:02,73167,CH5300772640237288398,Andrea,Pinana,6693,Broglio,TI,it,mtan


###Feature Engineering###

In [3]:
# Bestimmte Daten in das richtige Format konvertieren

# timestamp in Datums-und Zeitformat umwandeln
trx_data_ol.timestamp = pd.to_datetime(trx_data_ol['timestamp'])
trx_data_ol.valuta_date = pd.to_datetime(trx_data_ol['valuta_date'])
#Suche des Datensatzes nach Null-Werten
trx_data_ol.isnull().values.sum()
trx_data_ol.dtypes


timestamp            datetime64[ns]
paym_id                       int64
send_bc_nr                   object
amount                        int64
rcv_bc_nr                    object
rcv_bc_iban                  object
rcv_bc_country               object
rcv_bc_code                  object
rcv_iban                     object
valuta_date          datetime64[ns]
cust_vertrag_nr               int64
cust_konto_nr                object
cust_vname                   object
cust_nname                   object
cust_plz                      int64
cust_ort                     object
cust_kanton                  object
cust_sprachregion            object
cust_auth_device             object
dtype: object

**Encoding kategorische Werte in nummerische Werte**

In [4]:

#Encoding Daten mit Dtype Object in nummerische Werte
#trx_data_ol = pd.get_dummies(trx_data_ol, prefix_sep='_', drop_first=True)
trx_data_ol = pd.get_dummies(trx_data_ol, columns=['send_bc_nr', 'rcv_bc_nr', 'rcv_bc_iban',
                                    'rcv_bc_country','rcv_bc_code','rcv_iban',
                                    'cust_konto_nr','cust_vname','cust_nname',
                                    'cust_ort','cust_kanton','cust_sprachregion',
                                    'cust_auth_device'], drop_first=True)
trx_data_ol.head()

,timestamp,paym_id,amount,valuta_date,cust_vertrag_nr,cust_plz,rcv_bc_nr_AEBANK83XXX,rcv_bc_nr_ALBANK11XXX,rcv_bc_nr_ATBANK60XXX,rcv_bc_nr_AZBANK13XXX,...,cust_ort_Wünnewil,cust_ort_Zell LU,cust_kanton_AR,cust_kanton_FR,cust_kanton_GE,cust_kanton_LU,cust_kanton_TI,cust_sprachregion_fr,cust_sprachregion_it,cust_auth_device_ptan
0,2018-01-01 08:00:00,10000010,15368,2018-01-01 08:00:00,76311,6020,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2018-01-01 08:00:00,10000011,89137,2018-01-01 20:00:00,69891,9056,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,2018-01-01 08:01:00,10000012,87673,2018-02-01 08:01:00,78573,1253,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
3,2018-01-01 08:01:00,10000013,55941,2018-02-01 20:01:00,75443,6026,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2018-01-01 08:02:00,10000014,88173,2018-03-01 08:02:00,73167,6693,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [7]:
trx_data_ol.dtypes

timestamp                             datetime64[ns]
paym_id                                        int64
amount                                         int64
valuta_date                           datetime64[ns]
cust_vertrag_nr                                int64
cust_plz                                       int64
rcv_bc_nr_AEBANK83XXX                          uint8
rcv_bc_nr_ALBANK11XXX                          uint8
rcv_bc_nr_ATBANK60XXX                          uint8
rcv_bc_nr_AZBANK13XXX                          uint8
rcv_bc_nr_BABANK16XXX                          uint8
rcv_bc_nr_BEBANK15XXX                          uint8
rcv_bc_nr_BGBANK19XXX                          uint8
rcv_bc_nr_BHBANK14XXX                          uint8
rcv_bc_nr_BRBANK17XXX                          uint8
rcv_bc_nr_BYBANK84XXX                          uint8
rcv_bc_nr_CHBANK71XXX                          uint8
rcv_bc_nr_CRBANK20XXX                          uint8
rcv_bc_nr_CYBANK85XXX                         

**Datensatz in Trainings- und Testingdatensatz aufteilen**

Da wir über einen relative grosses Datenset verfügen können wir eine klassisches Spliting der Daten durchführen d.h. 80% (48'000)/20% (12'000) 


In [0]:
from sklearn.model_selection import train_test_split
tdo_train, tdo_test = train_test_split(trx_data_ol,test_size=0.2, random_state = 70)


In [0]:
print ('Grösse Trainingsdatesatz (Shape) = ', tdo_train.shape)
print ('Grösse Testdatensatz (Shape) =', tdo_test.shape)

Grösse Trainingsdatesatz (Shape) =  (48000, 86027)
Grösse Testdatensatz (Shape) = (12000, 86027)


###Modell Training###

Nachfolgenden werden wir ein neuronales Netzwerk definieren und dieses mit den oben bearbeiteten Daten trainieren.

Für den Aufbau des neuronalen Netzwerks werden ich die Machine Learning Library **TensorFlow** von Google verwenden.

**TensorFlow laden und Hyperparameter definieren**

**Erstellung des neuronalen Netzwerk-Modells**

https://www.youtube.com/watch?v=BhpvH5DuVu8&list=PLQVvvaa0QuDfKTOs3Keq_kaG2P55YRn5v&index=46



In [34]:
import tensorflow as tf

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 10
batch_size = 1000

x = tf.placeholder('float',[None, 784])

def neural_network_model(data):
  hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784,n_nodes_hl1])),
                   'biases':tf.Variable(tf.random_normal(n_nodes_hl1))}
  hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl2])),
                   'biases':tf.Variable(tf.random_normal(n_nodes_hl2))}
  hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3])),
                   'biases':tf.Variable(tf.random_normal(n_nodes_hl3))}
  output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3,n_nodes_hl1])),
                   'biases':tf.Variable(tf.random_normal(n_classes))}
  
  l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
  l1 = tf.nn.relu(l1)
  
  l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
  l2 = tf.nn.relu(l2)
  
  l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
  l3 = tf.nn.relu(l3)
                 
  output = tf.add(l3, output_layer['weights'])+ output_layer['biases']
  
  return output
                  
def train_neural_network(x):
  prediction = neural_network_model(x)
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction,y))
  optimizer = tf.train.AdamOptimizer().minimize(cost)
  
  hm_epochs = 10
                  
  with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    # Hier wird das Modell trainiert
    for epoch in hm_epochs:
        epoch_loss = 0
        for _ in range(int(tdo_train.train.num_examples/batch_size)):
            x = tdo_train.next_batch(batch_size)
            _, c = sess.run([optimizer, cost], feed_dict = {x: x})
            epoch_loss += c
        print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)
                  
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct,'float'))
    print('Accuracy:', accuracy.eval({x:tdo_test}))
    
train_neural_network(x)

ValueError: ignored

#Ermittlung potenzieller Kundenrisikogruppen# 
##Supervised Learning mit neuronalen Netzwerken##


In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Laden der 600000 Transaktionsdaten 
trx_data_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/trx_data_ml.csv'

trx_data_ml = pd.read_csv(trx_data_url, delimiter=';')
trx_data_ml.head()


In [0]:
trx_data_ol.nunique()

In [0]:
trx_data.fraud_id.map(lambda x:1 if x>=1 else 0).mean()

##Datensatz analysieren##

##Tipp: Plotting Learning Curves - Scikit !!!!

In [0]:
# Visualisierung der Nummerischen Daten in dem Datensatz
trx_data.hist(bins=50, figsize=(20,15))
plt.show()

In [0]:
# Suche nach den Korrelationen - Teil 1
corr_matrix = trx_data.corr()
print ('Korrelation zu Payment ID','\n',corr_matrix['paym_id'].sort_values(ascending=False))
print ('Korrelation zu Zahlungssumme','\n',corr_matrix['amount'].sort_values(ascending=False))
print ('Korrelation zu Kundenvertrag','\n',corr_matrix['cust_vertrag_nr'].sort_values(ascending=False))
print ('Korrelation zu Betrugsid','\n',corr_matrix['fraud_id'].sort_values(ascending=False))

In [0]:
# Bereinigung der Daten
pd.to_datetime(trx_data['timestamp'])
pd.to_datetime(trx_data['valuta_date'])
np.where(np.isnan(trx_data))

In [0]:
# Datensatz in Trainings(80%)- und Testdatenset (20%) teilen
from sklearn.model_selection import train_test_split
train_trx_data, test_trx_data = train_test_split(trx_data, test_size=0.2, random_state=42) 
train_trx_data.count(), test_trx_data.count()

In [0]:
# Dimensionsreduktion mit PCA
from sklearn.decomposition import PCA # Import der PCA Funktion von Scikit Learn
from sklearn import preprocessing # Die Preprocessing Funktion hilft uns die Daten zu skalieren bevor sie mit PCA verwendet werden.